<h6 align=right> 🐍 Python akademie - lekce 10 - 03.05.2023</h6>

<br>

# <h1 align=center><font color=black size=24><b> 10_01 🪲 Debugging</font></h1>

<br>


<br>

---

### **Zajímavé odkazy z této lekce:**


* 
* 



---

<br>

Proces *debugování* je označení, které popisuje odstraňování chyb.

Původ slova *ladění*, najdeš u vědkyně [Grace Hopper](https://en.wikipedia.org/wiki/Debugging).

V Pythonu to prakticky vypadá tak, že v programu či zápise snažíš:
1. Naleznout místo, které **vyvolává výjimku**,
2. naleznout místo, které **způsobuje neočekávané chování**.

In [1]:
def formatuj_jmeno(string, symbol: str = "."):
    """
    :Example:
    >>> formatuj_jmeno("marek.parek")
    'Marek'
    """
    jen_jmeno = string.split(symbol)  
    return jen_jmeno.title() 

In [2]:
def vytvor_pozdrav(jmeno: str) -> str:
    """
    :Example:
    >>> vytvor_pozdrav("marek.parek")
    'Toto je Marek, zdravíme!'
    """
    return " ".join(("Toto je", "".join(formatuj_jmeno(jmeno)), "zdravíme!"))

In [3]:
print(vytvor_pozdrav("petr.svetr"))

AttributeError: 'list' object has no attribute 'title'

Co nám *interpret* vypsal:
* `AttributeError`, tedy typ výjimky, kterému interpret chybu přiřadil,
* `Traceback`, postup, odkud chybu stopoval,
* `<v_cem> in <kde>`, prostředí, kde se chyba projevila,
* `'list' object has no attribute 'title'`, popisek výjimky.

Vidíš tedy, kde všude *interpret* prošel tvůj zápis a co se mu *nepozdávalo*.

<br>

## **Debugování**

---

Nejlepší, co na začátek můžeš provést, je udělat si v situaci jasno.

Vědět, s čím máš tu čest.

K tomu ti pomůže hned několik zabudovaných funkcí:
* `print`,
* `type`,
* `dir`,
* `vars`,
* `locals`,
* `globals`.



#### **print**

Pomocí **jednoduchého výstupu** uvidíš, s čím vůbec pracuješ:

In [4]:
def formatuj_jmeno(string, symbol: str = "."):
    """
    :Example:
    >>> rozdel_string("marek.parek")
    'Marek'
    """
    jen_jmeno = string.split(symbol)
    print(jen_jmeno)                  # doplněná zab. funkce
    return jen_jmeno.title() 

In [5]:
vytvor_pozdrav("petr.svetr")

['petr', 'svetr']


AttributeError: 'list' object has no attribute 'title'

Funkce `vars`, `locals`, `globals` tedy není třeba aplikovat.

Ze zápisu je patrné, že pracuješ **se správným objektem**.

<br>

#### **type**


Když vidíš, jak hodnoty vypadají, je potřeba ověřit, jak je zpracuje *interpret*:

In [6]:
def formatuj_jmeno(string, symbol: str = "."):
    """
    :Example:
    >>> rozdel_string("marek.parek")
    'Marek'
    """
    jen_jmeno = string.split(symbol)
    print(type(jen_jmeno))            # doplněná zab. funkce
    return jen_jmeno.title() 

In [7]:
print(vytvor_pozdrav("petr.svetr"))

<class 'list'>


AttributeError: 'list' object has no attribute 'title'

Teď, když znáš datový typ, můžeš zkontrolovat, jestli pracuješ **se správnou metodou**.

#### **dir**


Pomocí zab. funkce `dir` ověříš, které metody máš **pro konkrétní objekt** k dispozici:

In [8]:
def formatuj_jmeno(string, symbol: str = "."):
    """
    :Example:
    >>> rozdel_string("marek.parek")
    'Marek'
    """
    jen_jmeno = string.split(symbol)
    print(dir(jen_jmeno))            # doplněná zab. funkce
    return jen_jmeno.title()         # metoda, kterou ve výstupu hledám

In [9]:
print(vytvor_pozdrav("petr.svetr"))

['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


AttributeError: 'list' object has no attribute 'title'

Vidíš, že použitou metodu skutečně ve výstupu nenajdeš.

Takže hodnoty, které zpracováváš a metody, které k tomu používáš, **nejsou kompatibilní**:

In [10]:
cele_jmeno = ['petr', 'svetr']

In [11]:
print(cele_jmeno[0].title())

Petr


In [12]:
print(cele_jmeno[3].title())

IndexError: list index out of range

V tento okamžik dovedeš celou situaci **jednoduše replikovat**.

To samozřejmě není vždy nutné, ale obzvlášť ze začátku, pomáhá spojovat souvislosti.

In [13]:
def formatuj_jmeno(string, symbol: str = "."):
    """
    :Example:
    >>> rozdel_string("marek.parek")
    'Marek'
    """
    try:
        jen_jmeno = string.split(symbol)[0]
        
    except IndexError:
        vystup = string
        print("Nelze indexovat")    
    except AttributeError:
        vystup = string
        print("Nelze použít zadané metody")
    else:
        vystup = jen_jmeno.title()
        print("Ukládám zadanou hodnotu...")
    finally:
        return vystup

In [14]:
print(vytvor_pozdrav("petr.svetr"))

Ukládám zadanou hodnotu...
Toto je Petr zdravíme!


<br>

## **Debugování, náročnější situace**

---

Pokud je problém náročnější, pro odhalení budeš potřebovat silnější nástroj:
* knihovna `pdb` (není intuitivní, ale můžeš jej použít všude),
* **debugger** součástí editoru/IDE (graficky pohodlné ovládání, ne vždy dostupné).

##### Demo: pomocná úloha

Napiš funkci `projdi_vsechny_udaje`, která prochází různé množství zadaných stringů. Procházej pomocí této funkce tak dlouho, dokud nenarazíš na řádek obsahující string `'end'`.
Potom proces zastav.

Tyto stringy jsou emailové adresy, ze kterých chceš vypsat jméno, příjmení a doménu. Pomocí funkce `rozdel_email` získej vždy první část ze stringu a zbytek. Obě části vrať.

In [15]:
def projdi_vsechny_udaje(*args) -> None:
    for zaznam in args:
        if "end" in zaznam:  # if zaznam == "end":
            break
        else:
            jmeno, zbytek = rozdel_email(zaznam, ".")
            prijmeni, domena = rozdel_email(zbytek, "@")
            print(
                {"jmeno": jmeno, "prijmeni": prijmeni, "domena": domena}
            )

In [16]:
def rozdel_email(email: str, symbol: str) -> tuple:
    prvni_cast, zbytek = email.split(symbol, maxsplit=1)
    return (prvni_cast.title(), zbytek)

In [ ]:
projdi_vsechny_udaje(
    'petra.fulinova@firma.cz',
    'adela.vancurova@firma.cz',
    'andrea.hertlova@firma.cz',
    'petr.vyhnis@firma.cz',
    'jan.feckanin@firma.cz',
    'pavel.harant@firma.cz',
    'zdenka.bendova@firma.cz',
    'monika.miczova@firma.cz',
    'jan.mosquito@firma.cz',
    'barbora.suvova@firma.cz',
    'lenka.kafkova@firma.cz',
    'nikola.hoffmannova@firma.cz',
    'daniela.sedlakova@firma.cz',
    'ivana.jerabkova@firma.cz',
    'valeria.jagerska@firma.cz',
    'hana.bayerova@firma.cz',
    'tomas.zamecnik@firma.cz',
    'helena.strasilova@firma.cz',
    'jana.kralova@firma.cz',
    'hermina.duskova@firma.cz',
    'dana.mirgova@firma.cz',
    'end',
    '...'
)

Po spuštění se ale stalo něco podivného.

Vidíš, že funkce neprošly všechny zadané hodnoty.

Protože se neobjevila **žádná výjimka**, je potřeba debugovat sofitikovanějšími způsoby:
1. Knihovna `pdb`,
2. Pycharm, debugger.
